In [1]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [2]:
driver_path = 'C:/Users/MY/Downloads/chromedriver.exe'

In [3]:
def biz_khan(query,driver_path):
    domain = '한국일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
    url_domain = 'https://search.naver.com/search.naver?where=news&query='
    news_domain ='%EB%89%B4%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1469&nso=&is_sug_officeid=0' 
    url = url_domain + query + news_domain
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    # 기사 url을 list형태로 반환
    news_list = soup.select('#main_pack > section > div > div.group_news > ul')
    url_list = []
    news_list = news_list[0].find_all('li')

    for url in news_list:
        url_list.append(str(url.find_all('a')[0].get('data-url')))
    driver.close()
    
    data_list = []
    head_list = []
    for url in url_list:
        driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
        time.sleep(2)
        driver.get(url)
        time.sleep(2)
        # try문 안에서 각자 언론사에 맞게 수정
        try:
            soup = BeautifulSoup(driver.page_source,'html.parser')
            head_list.append(soup.find('h2').text)
            tags = soup.select_one('body > div.wrap > div.container.end.end-uni > div.end-body > div > div.col-main.read').find_all('p')
            data = [x.text for x in tags]
            data_list.append(' '.join(data))
        except:
            print('None_type')
        driver.close()
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

biz = biz_khan('삼성전자 슬로건',driver_path)

In [4]:
biz.to_csv('한국일보.csv',encoding='CP949',index=None)

In [5]:
data1 = pd.read_csv('한국일보.csv',encoding='CP949')

In [6]:
data1

,언론사,제목,내용
0,한국일보,"삼성 배터리·LG 태양광, 에너지 미래 밝힌다","삼성SDI·제일모직 합병 시너지, 전기차·휴대폰 배터리 경쟁력 강화 LG 태양광 부..."
1,한국일보,눈시울 붉힌 손학규 “국민 승리 때까지 멈추지 않겠다”,\n“대통령 하면 제일 잘 할 것 같았는데...안철수 후보가 한 풀어달라” \n \...
2,한국일보,"평창 성화, 7500명 주자가 101일 동안 2,018km 불 밝힌다",‘모두를 빛나게 하는 불꽃(Let Everyone Shine).’ 2018년 평창 ...
3,한국일보,제2의 알파고 만든다…국내 기업들 AI 개발 잰걸음,인공지능 시장을 선점하기 위한 국내 기업들의 경쟁이 달아오르고 있다. 지난 3월 구...
4,한국일보,"[브리핑 종합] 알바몬, 임금체불 업체 211개 공개 外","\n알바몬, 임금체불 업체 211개 공개\n 취업포털 잡코리아가 운영하는 아르바이트..."
5,한국일보,나에게 ‘♥7’이란…,‘가슴 벅찬’ ‘취향저격’ ‘기대되는’ ‘매력 철철’…. 똑 같은 이미지라도 읽는...
6,한국일보,"[브리핑] 현대차그룹, 광주 ‘전통시장 활성화 프로젝트’ 시동 外","\n현대차그룹, 광주 ‘전통시장 활성화 프로젝트’ 시동\n 현대자동차그룹은 광주창조..."
7,한국일보,“기초화장품을 만나는 하루 5분 남짓이라도 행복하길~”,‘노드메이슨’ 이병훈 대표 인터뷰 최근 뷰티 트렌드세터들의 ‘잇(it) 아이템’으...
8,한국일보,“청년들 겁없이 방아쇠 당기도록… 초기 스타트업 지원에 집중을”,\n국내 대기업 공격적 경영 꺼려\n \n주력사업 대체할 먹거리 못찾아\n \n스타...
9,한국일보,갤S6 열흘간 국내 판매량 20만대… 예약판매 30만대에도 못미쳐,삼성전자의 전략 스마트폰 갤럭시S6와 갤럭시S6엣지 판매량이 ‘대박론’에도 불구하고...
